In [9]:
import os
import pydicom
import numpy as np
from tqdm import tqdm
from PIL import Image
import pandas as pd
import shutil

In [10]:
import pandas as pd
import os
import shutil

# Load dataset
df = pd.read_csv("train.csv")

mapped_labels = [
    "No finding",
    "Cardiomegaly",
    "Pleural effusion",
    "Consolidation",
    "Atelectasis",
    "Pneumothorax",
    "Lung Opacity",
    "Nodule/Mass"
]

df_filtered = df[df["class_name"].isin(mapped_labels)]

# Count occurrences of each disease and get the top 6
disease_counts = df_filtered["class_name"].value_counts()
top_diseases = disease_counts.nlargest(6)

# Save the disease names and counts to a text file
top_diseases.to_csv("dataset_six_images.txt", header=True, index=True)
print("Saved top 6 diseases to 'dataset_six_images.txt'.")

Saved top 6 diseases to 'dataset_six_images.txt'.


In [12]:

top_disease_df = df_filtered[df_filtered["class_name"].isin(top_diseases.index)]
top_disease_df = top_disease_df.groupby("class_name").apply(lambda x: x.sample(n=min(1000, len(x)), random_state=42))
top_disease_df = top_disease_df.reset_index(drop=True)

dicom_dir = "train/"  # Original .dicom files
subset_dicom_dir = "new_six_subset_dicom/"  # Folder for selected .dicom files
os.makedirs(subset_dicom_dir, exist_ok=True)

# only images that exist in the folder are considered
existing_images = set(os.listdir(dicom_dir))
top_disease_df["filename"] = top_disease_df["image_id"] + ".dicom"
top_disease_df = top_disease_df[top_disease_df["filename"].isin(existing_images)]

for file in top_disease_df["filename"].unique():
    shutil.copy(os.path.join(dicom_dir, file), subset_dicom_dir)

print(f"Successfully moved {len(os.listdir(subset_dicom_dir))} DICOM images for the top 6 most common diseases.")

/var/folders/zf/cz8p4qss5g104j1pq2n096y80000gn/T/ipykernel_43704/1498644194.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_disease_df = top_disease_df.groupby("class_name").apply(lambda x: x.sample(n=min(1000, len(x)), random_state=42))


Successfully moved 3204 DICOM images for the top 6 most common diseases.


**Reformat DICOM into JPG**

In [8]:
print(top_disease_df["class_name"].value_counts())

class_name
Nodule/Mass         998
Lung Opacity        993
Cardiomegaly        992
Pleural effusion    991
No finding          990
Consolidation       551
Name: count, dtype: int64


In [13]:

subset_jpg_dir = "vinbigdata_subset_jpg/"
os.makedirs(subset_jpg_dir, exist_ok=True)

for file in tqdm(os.listdir(subset_dicom_dir)):
    if file.endswith(".dicom"):
        dicom_path = os.path.join(subset_dicom_dir, file)
        jpg_path = os.path.join(subset_jpg_dir, file.replace(".dicom", ".jpg"))

        try:
            # Force Load DICOM
            dicom_img = pydicom.dcmread(dicom_path, force=True)

            # Force Decompression if needed
            if dicom_img.file_meta.TransferSyntaxUID.is_compressed:
                dicom_img.decompress()

            img_array = dicom_img.pixel_array.astype(np.float32)
            img_array = (img_array - img_array.min()) / (img_array.max() - img_array.min()) * 255.0
            img_array = img_array.astype(np.uint8)

            img = Image.fromarray(img_array)
            img.convert("L").save(jpg_path)  

        except Exception as e:
            print(f"Error processing {file}: {e}")

print(f"Converted {len(os.listdir(subset_jpg_dir))} DICOM images to JPG")


  0%|          | 0/3204 [00:00<?, ?it/s]/opt/homebrew/anaconda3/lib/python3.11/site-packages/pydicom/valuerep.py:440: UserWarning: Invalid value for VR UI: 'f1a45afaee0efd07fef17057f3942464'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warn_and_log(msg)
  0%|          | 1/3204 [00:00<43:23,  1.23it/s]/opt/homebrew/anaconda3/lib/python3.11/site-packages/pydicom/valuerep.py:440: UserWarning: Invalid value for VR UI: 'ad21e3f856bd69daf2ee8b6699bd32cf'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warn_and_log(msg)
  0%|          | 3/3204 [00:01<22:35,  2.36it/s]/opt/homebrew/anaconda3/lib/python3.11/site-packages/pydicom/valuerep.py:440: UserWarning: Invalid value for VR UI: '774c1ca3798d3054ac1a413570e665c9'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for

Converted 3204 DICOM images to JPG


**Resize the images:**

In [14]:

image_folder = "vinbigdata_subset_jpg"
processed_folder = "vinbigdata_resized"
os.makedirs(processed_folder, exist_ok=True)


for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)
    save_path = os.path.join(processed_folder, img_name)

    try:
        # grayscale
        img = Image.open(img_path).convert("L")
        img = img.resize((224, 224))
        img.save(save_path)  
    except Exception as e:
        print(f"Error processing {img_name}: {e}")

print(f"Resized images saved to '{processed_folder}'")


Resized images saved to 'vinbigdata_resized'
